In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU
import string
from string import digits
import re
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
path1 = (r"C:\Users\91880\Desktop\Intern\dcep.eng.txt.en")
path2 = (r"C:\Users\91880\Desktop\Intern\dcep.en-fr_fr.txt.fr")

In [ ]:
lines= pd.read_table(path1, names=['eng'])
lines2 = pd.read_table(path2,names =['fr'])
lines = lines.join(lines2['fr'])
lines = lines[0:10000]
arr = lines.values.tolist()

In [ ]:
lines.shape

In [ ]:
lines.head()

In [ ]:
from nltk.tokenize import word_tokenize
import re
import string
import unicodedata
input_texts = []
target_inputs = []
target_outputs = []

def unicode_to_ascii(s):
     return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

for line in arr:
    line[0]=re.sub(r"[^a-zA-Z0-9]+", ' ', str(line[0]))
    line[1]=re.sub(r"[^a-zA-Z0-9]+", ' ', str(line[1]))
    line[0] = re.sub("(\d+)","",str(line[0]))
    line[1] = re.sub("(\d+)","",str(line[1]))
    input_text = unicode_to_ascii(str(line[0]).lower().strip())
    input_text = input_text.translate(str.maketrans('', '', string.punctuation))
  
    target_text = unicode_to_ascii(str(line[1]).lower().strip())
    target_text = target_text.translate(str.maketrans('', '', string.punctuation))
    target_input = "<bof>" + target_text 
    target_output = target_text+"<eos>"
    input_texts.append(input_text)
    target_inputs.append(target_input)
    target_outputs.append(target_output)
print(len(input_texts))
print(len(target_outputs))
print(len(target_inputs))
print(input_texts[14])
print(target_outputs[14])
print(target_inputs[14])

In [ ]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences
#convert sentences to word tokens and find length of vocab and max sentence length for input and output
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
Source_vocabulary = input_tokenizer.word_index.keys()

num_source_tokens = len(input_tokenizer.word_index)
max_source_seq_length = max(len(text_to_word_sequence(x)) for x in input_texts)


output_tokenizer = Tokenizer()
output_tokenizer.fit_on_texts(target_inputs + target_outputs)
target_vocabulary = output_tokenizer.word_index.keys()

num_target_tokens = len(output_tokenizer.word_index)
max_target_seq_length = max(len(text_to_word_sequence(y)) for y in target_inputs)

In [ ]:
print('Number of samples:', len(input_texts))
print('Number of unique source language tokens:', num_source_tokens)
print('Number of unique target language tokens:', num_target_tokens)
print('Max sequence length of source language:', max_source_seq_length)
print('Max sequence length of target language:', max_target_seq_length)

In [ ]:
#convert unique input words to interger values
num_words_output = num_target_tokens + 1
input_int = input_tokenizer.texts_to_sequences(input_texts)
target_output_int = output_tokenizer.texts_to_sequences(target_outputs)
target_input_int = output_tokenizer.texts_to_sequences(target_inputs)

In [ ]:
#pad the sequences to the same length
encoder_input = pad_sequences(input_int, maxlen=max_source_seq_length)
print("endocder input shape", encoder_input.shape)

#add padding for encoder after the sentence integers
decoder_input = pad_sequences(target_input_int, maxlen=max_target_seq_length, padding='post')
print("decoder input shape", decoder_input.shape)


decoder_output = pad_sequences(target_output_int, maxlen=max_target_seq_length, padding='post')
print("decoder output shape", decoder_output.shape)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embedding = dict()
glove_file="C:\\Users\\91880\\Desktop\\Intern\\glove.6B.50d.txt"

g = open(glove_file , 'r',encoding="utf8")


for l in g:
    vects = l.split()
    w = vects[0]
    vect_dim = asarray(vects[1:], dtype='float32')
    embedding[w] = vect_dim

g.close()

In [ ]:
words =  num_source_tokens+1
embed_mat = zeros((words, 50))
for value, position in input_tokenizer.word_index.items():
    embedding_vect = embedding.get(value)
    if embedding_vect is not None:
        embed_mat[position] = embedding_vect

In [ ]:
print(embed_mat.shape)

In [ ]:
print(embedding['i'])
print(embed_mat[1])

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

In [ ]:
from keras.layers import Embedding
with strategy.scope():
    
    layer_embedding = Embedding(words, 50, weights=[embed_mat], input_length=max_source_seq_length)
    
    
    nodes_lstm = 256
    epochs = 100
    
    
    encoder_inputs = Input(shape=(max_source_seq_length,))
    x = layer_embedding(encoder_inputs)
    encoder_lstm = LSTM(nodes_lstm, return_state=True,return_sequences=True,dropout=0.4,recurrent_dropout=0)
    encoder_outputs1, state_h, state_c = encoder_lstm(x)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(max_target_seq_length-1,))
    decoder_embedding = Embedding(num_words_output, nodes_lstm)
    decoder_inputs_x = decoder_embedding(decoder_inputs)
 

    decoder_lstm = LSTM(nodes_lstm, return_state=True,return_sequences=True,dropout=0.4,recurrent_dropout=0)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)


    decoder_dense = Dense(num_words_output, activation='sigmoid')
    decoder_outputs = decoder_dense(decoder_outputs)



    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#train-test split 85:15 for training and test for each layer training data inputted into model fit
from sklearn.model_selection import train_test_split
encoder_train, encoder_test, decoder_train, decoder_test,decoder_targets_train, decoder_targets_test = train_test_split(encoder_input, decoder_input,decoder_output, test_size=0.15, random_state=42)


In [ ]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((encoder_train, decoder_train[:,:-1]))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((encoder_test, decoder_test[:,:-1]))

In [ ]:
train_tf_dataset_1 = tf.data.Dataset.from_tensor_slices(decoder_train.reshape(decoder_train.shape[0],decoder_train.shape[1], 1)[:,1:])
test_tf_dataset_1 = tf.data.Dataset.from_tensor_slices(decoder_test.reshape(decoder_test.shape[0],decoder_test.shape[1], 1)[:,1:])

In [ ]:
print(decoder_test[0])
print(decoder_test[1])

In [ ]:
train_dataset = tf.data.Dataset.zip(((train_tf_dataset, train_tf_dataset_1)))

train_dataset.element_spec

In [ ]:
test_dataset = tf.data.Dataset.zip(((test_tf_dataset, test_tf_dataset_1)))

test_dataset.element_spec

In [ ]:
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
len(train_dataset)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = "intern_ml-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

es = EarlyStopping(monitor = 'val_loss'  ,mode='min' , verbose=1 , patience=2)
model.fit(train_dataset,validation_data=test_dataset, epochs=20 ,verbose = 1,callbacks=[es] , batch_size = BATCH_SIZE)
model.save('seq2seq_source_target.h5')

In [ ]:
model.load_weights('seq2seq_source_target.h5')

In [ ]:
#update encoder and decoder models for testing to allow predictions word by word
#encoder stays the same
encoder_model = Model(encoder_inputs, encoder_states)

decoder_h = Input(shape=(nodes_lstm,))
decoder_c = Input(shape=(nodes_lstm,))
decoder_states = [decoder_h, decoder_c]

#shape is changed to single decoders rather than full phrase, can iterate through and translate sentence
single_dec = Input(shape=(1,))
single_dec_x = decoder_embedding(single_dec)
decoder_out, h, c = decoder_lstm(single_dec_x, initial_state=decoder_states)

decode = [h, c]
decoder_out = decoder_dense(decoder_out)

decoder_model = Model([single_dec] + decoder_states,[decoder_out] + decode)

In [ ]:
#convert integers back to words for source and target
#create dictionaries to convert them back
word_input = {value:key for key, value in input_tokenizer.word_index.items()}
word_target = {value:key for key, value in output_tokenizer.word_index.items()}

#method used to translate, converting the integer values of each sentence back to the word vectors
def translator(input):
    states = encoder_model.predict(input)
    target = np.zeros((1, 1))
    
    target[0, 0] = output_tokenizer.word_index['bof']
    eos = output_tokenizer.word_index['eos']
    out_sen = []

    for _ in range(max_target_seq_length):
        out_tok, h, c = decoder_model.predict([target] + states)
        index = np.argmax(out_tok[0, 0, :])

        if eos == index:
            break

        word = ''

        if index > 0:
            word = word_target[index]
            out_sen.append(word)

        target[0, 0] = index
        states = [h, c]

    return ' '.join(out_sen)

In [ ]:
#BLEU evalution of test dataset

import nltk
from statistics import mean 
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4
accuracy_overall = []
#iterate through all test set data and predict french translation
#record the accuracy for each sentence and add to list, get the mean value of this list 
#returns the accuracy of test data
for sentence in range (20,51):
    input_sequence = encoder_input[sentence:sentence+1]
    translate = translator(input_sequence)
    #   print('English', input_texts[sentence])
    #   print('French', translate)
    #   print("------------------------------------------------")


    actual = target_outputs[sentence].split()
    predict = translate.split()
    actual = actual[:-1]
          #print(actual,predict)

          #use BLEU to score the actual vs predicted sets
    accuracy = nltk.translate.bleu_score.sentence_bleu([actual],predict , smoothing_function = smoothie)
    if len(actual)>=4:
        accuracy_overall.append(accuracy)

#return the accuracy from the average across all translations
test_accuracy = round(mean(accuracy_overall),2)
print("Test Score is: ", test_accuracy)

In [ ]:
test_accuracy = round(mean(accuracy_overall),2)
print("Test Score is: ", test_accuracy)